In [ ]:
import os
import pandas as pd
import requests

# Caminho do arquivo CSV de entrada e saída
input_csv_path = r'xxxx'
output_csv_path = r'xxxx'

# Verificar se o arquivo CSV de entrada existe
if not os.path.exists(input_csv_path):
    print(f'Arquivo CSV de entrada não encontrado em: {input_csv_path}')
    exit()

# Lista de encodings a serem tentados
encodings = ['utf-8', 'latin-1', 'iso-8859-1']

df_indicadores = None
for encoding in encodings:
    try:
        df_indicadores = pd.read_csv(input_csv_path, sep=';', encoding=encoding)
        print(f'Cabeçalho do arquivo CSV:\n{df_indicadores.head()}')  # Imprime o cabeçalho para verificação
        break  # Se conseguiu ler, interrompe o loop
    except UnicodeDecodeError:
        print(f'Erro ao ler arquivo CSV com encoding {encoding}. Tentando próximo encoding...')
    except Exception as e:
        print(f'Erro desconhecido ao ler arquivo CSV: {e}')

if df_indicadores is None:
    print(f'Não foi possível ler o arquivo CSV {input_csv_path} com os encodings fornecidos.')
    exit()

# Função para buscar dados de um indicador do BACEN
def get_bacen_data(series_id):
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{series_id}/dados?formato=json'
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lança exceção para erros HTTP

        if response.status_code == 200:
            data = response.json()
            if not data:
                print(f'Dados vazios recebidos para o indicador {series_id}.')
                return None
            
            df = pd.DataFrame(data)
            
            # Verifique a estrutura dos dados
            if 'valor' not in df.columns:
                print(f"A coluna 'valor' não foi encontrada para o ID {series_id}.")
                return None
            
            # Ajustar os dados
            df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
            df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
            df = df.sort_values('data')
            
            # Filtrar dados após o ano 2000
            df = df[df['data'].dt.year >= 2000]
            
            return df
        else:
            print(f'Falha ao buscar dados para o indicador {series_id}. Status Code: {response.status_code}')
            return None
        
    except requests.exceptions.RequestException as e:
        print(f'Erro ao fazer requisição HTTP para o indicador {series_id}: {e}')
        return None
    except Exception as e:
        print(f'Erro desconhecido ao buscar dados para o indicador {series_id}: {e}')
        return None

# Função para obter a unidade e periodicidade de um indicador específico
def get_unidade_periodicidade(series_id):
    # Exemplo simples de mapeamento para fins ilustrativos
    unidades = {
        11: {'unidade': '% a.a.', 'periodicidade': 'Diária'},
        188: {'unidade': 'Índice', 'periodicidade': 'Mensal'},
        189: {'unidade': 'Índice', 'periodicidade': 'Mensal'},
        # Adicionar mais mapeamentos conforme necessário
    }
    
    if series_id in unidades:
        return unidades[series_id]
    else:
        return {'unidade': 'Desconhecida', 'periodicidade': 'Desconhecida'}

# Inicialização de uma lista para armazenar os DataFrames de saída
data_frames = []

# Loop através dos indicadores do arquivo CSV
for index, row in df_indicadores.iterrows():
    series_id = row['codigo']  # Supondo que a coluna com o código do indicador seja 'codigo'
    
    # Ajuste para obter o nome do indicador corretamente
    nome_indicador = row['nome']  # Supondo que a coluna com o nome do indicador seja 'nome'
    
    # Buscar dados do BACEN para o indicador atual
    print(f'Buscando dados para o indicador: {nome_indicador} (código: {series_id})')
    df = get_bacen_data(series_id)
    
    if df is not None:
        # Obter unidade e periodicidade do CSV de entrada
        unidade = row['Unid.'] if 'Unid.' in row else 'Desconhecida'
        periodicidade = row['Per.'] if 'Per.' in row else 'Desconhecida'
        
        # Adicionar metadados ao DataFrame
        df['codigo'] = series_id
        df['nome_completo'] = nome_indicador
        df['unidade'] = unidade
        df['periodicidade'] = periodicidade
        
        data_frames.append(df)
    else:
        print(f'Nenhum dado encontrado para o indicador: {nome_indicador} (código: {series_id})')

# Verificar se algum dado foi coletado
if data_frames:
    # Concatenar todos os DataFrames
    final_df = pd.concat(data_frames, ignore_index=True)
    
    # Selecionar colunas específicas
    final_df = final_df[['data', 'valor', 'codigo', 'nome_completo', 'unidade', 'periodicidade']]
    
    # Formatar a coluna 'valor' para usar vírgula como separador decimal
    final_df['valor'] = final_df['valor'].apply(lambda x: f"{x:.2f}".replace('.', ','))
    
    # Adicionar a coluna 'moeda relacionada' com valor 'BRL'
    final_df['moeda relacionada'] = 'BRL'

    # Salvar os dados no arquivo CSV de saída
    try:
        final_df.to_csv(output_csv_path, index=False, sep=';', encoding='utf-8-sig')
        print(f'Dados salvos em: {output_csv_path}')
    except Exception as e:
        print(f'Erro ao salvar dados no arquivo CSV de saída: {e}')

else:
    print('Nenhum dado foi coletado.')
    
    
    

import os
import pandas as pd
from datetime import datetime

# Caminho do arquivo CSV de entrada e saída
input_csv_path = r'xxxx'

# Verificar se o arquivo CSV de entrada existe
if not os.path.exists(input_csv_path):
    print(f'Arquivo CSV de entrada não encontrado em: {input_csv_path}')
    exit()

# Ler o arquivo CSV de saída
try:
    df = pd.read_csv(input_csv_path, sep=';', encoding='utf-8-sig')
    print(f'Arquivo lido com sucesso. Cabeçalho:\n{df.head()}')
except Exception as e:
    print(f'Erro ao ler o arquivo CSV: {e}')
    exit()

# Garantir que a coluna 'data' esteja no formato datetime
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce')

# Criar novo DataFrame com os códigos e nomes completos
summary_df = df.groupby(['codigo', 'nome_completo']).agg(
    Data_do_ultimo_dado=('data', 'max')  # Última data com dado
).reset_index()

# Adicionar coluna de data de atualização
summary_df['Data_de_atualizacao_da_base'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Selecionar colunas para o novo CSV
summary_df = summary_df[['codigo', 'nome_completo', 'Data_do_ultimo_dado', 'Data_de_atualizacao_da_base']]

# Definir caminho para o novo arquivo CSV
output_summary_csv_path = os.path.join(os.path.dirname(input_csv_path), 'BACEN_resumo.csv')

# Salvar o resumo em um novo arquivo CSV
try:
    summary_df.to_csv(output_summary_csv_path, index=False, sep=';', encoding='utf-8-sig')
    print(f'Dados resumidos salvos em: {output_summary_csv_path}')
except Exception as e:
    print(f'Erro ao salvar dados resumidos no arquivo CSV: {e}')




Erro ao ler arquivo CSV com encoding utf-8. Tentando próximo encoding...
Cabeçalho do arquivo CSV:
   codigo                                               nome   Unid. Per.
0    1207              Produto interno bruto em R$ correntes      R$    A
1    1209  Produto interno bruto per capita em R$ corrent...      R$    A
2    1213  PIB trimestral (1990=100) - Dados observados -...  Índice    T
3    1214  PIB trimestral (1990=100) - Dados observados -...  Índice    T
4    1215  PIB trimestral (1990=100) - Dados observados -...  Índice    T
Buscando dados para o indicador: Produto interno bruto em R$ correntes (código: 1207)
Buscando dados para o indicador: Produto interno bruto per capita em R$ correntes_Desativada (código: 1209)
Buscando dados para o indicador: PIB trimestral (1990=100) - Dados observados - Lavouras (código: 1213)
Buscando dados para o indicador: PIB trimestral (1990=100) - Dados observados - Produção animal (código: 1214)
Buscando dados para o indicador: PIB trimestral 